# Investigating water quality dynamics in adaptive distribution networks with dynamic topology and control

Prepared by: Bradley Jenks\
Last updated: July 2024


### Preamble
This notebook investigates the impact of hydraulic control on temporal disinfectant variability in a large-scale operational network. This is performed through the following tasks:
1) Present case study network data and hydraulic control scenarios
2) Compute flow variability metrics to visualise hydraulic dynamics
3) Simulate disinfectant residuals under the considered control scenarios
4) Validate impacts of dynamic connectivity and hydraulic control through sensor data

The main outcome of this study is to evaluate if the considered hydraulic controls lead to increased disinfectant variability through changes in source water and possibly increased water quality dynamics. The flow variability metrics aim to inform areas in the network which stand to benefit from increased observability through sensor installations and/or manual grab sampling. Furthermore, insights from this study motivate the need for increased controllability of water quality dynamics in adaptive networks.

### Bristol Water Field Lab
#### Network description
The Bristol Water Field Lab network (BWFLnet) is a unique smart water demonstrator jointly operated by Imperial College London, Bristol Water Plc, and Cla-Val Ltd. It comprises two adjacent district metered areas (DMAs), which have been hydraulically connected via dynamic boundary valves. The network layout and properties are illustrated below using the `wntr` package, which is a Python wrapper for the open-source EPANET modelling software.

In [ ]:
import os
from src.functions import *
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
from itertools import combinations

import wntr

net_name = 'bwfl_2022_05_hw.inp'
project_dir = os.getcwd()
data_path = os.path.join(project_dir, 'data/')

wdn = load_network_data(os.path.join(data_path, net_name))

# improve matplotlib image quality
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # remove when saving plots

Print network properties.

In [ ]:
print(f"Number of nodes = {wdn.net_info['nn']}")
print(f"Number of links = {wdn.net_info['np']}")
print(f"Number of sources = {wdn.net_info['n0']}")
print(f"Number of 15-minute time steps = {wdn.net_info['nt']}")

BWFLnet has conitinuous MetriNet water quality sensors installed at nine (9) locations across the network. These locations are highlighted in the network layout plot below.

In [ ]:
# sensor node data
sensor_names = ['node_2746', 'node_1811', 'node_2367', 'node_2747', 'node_1809', 'node_2673', 'node_2506', 'node_1773', 'node_1802']

# plot network layout
legend_labels = {'Inlet (source)': 'black', 'Sensor node': 'black'}
plot_network_layout(wdn, sensor_nodes=sensor_names, legend_labels=legend_labels, sensor_labels=True)

#### Control configurations
In this study, we consider three (3) hydraulic control configurations for investigating water quality dynamics. These configurations employ the following valve types: (i) pressure reducing valves (PRVs); (ii) dynamic boundary valves (DBVs); and automatic flushing valves (AFVs).

1. **Single-feed DMAs.** This scenario considers a fixed topology with kept-shut isolation valves (IVs) separating the two DMAs. Furthermore, there is fixed-outlet pressure control at the DMA inlets. 

In [ ]:
# no control valves
prv = ['link_2756', 'link_2729', 'link_2743']
dbv = ['link_2747', 'link_2748']
iv_open = ['link_2320']
iv_close = ['link_2236', 'link_2682', 'link_2431', 'link_2389', 'link_2508', 'link_2555']

# find connecting nodes
prv_nodes = []
for link_id in prv:
    prv_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())
    
dbv_nodes = []
for link_id in dbv:
    dbv_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())
    
iv_close_nodes = []
for link_id in iv_close:
    iv_close_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())

# plot network layout
legend_labels = {'Inlet (source)': 'black', 'IV': 'black'}
plot_network_layout(wdn, prv_nodes=prv_nodes, iv_nodes=iv_close_nodes+dbv_nodes, legend_labels=legend_labels)

2. **Average zone pressure control.** This configuration represents the existing controls in BWFLnet, as operated by Bristol Water. PRVs are in the same locations as the single-feed DMA configuration, but they now have dynamic set-points at each 15-minute control interval. Additionally, two of the kept-shut IVs are replaced with remote-controlled throttle control valves, referred to as dynamic boundary valves (DBVs). These are programmed to shut during the overnight period for leakage management purposes and open during the day to convey flow between DMAs. The following paper describes this control configuration in more detail.
    * Wright, R., Stoianov, I., Parpas, P., Henderson, K., and King, J. (2015). 'Adaptive water distribution networks with dynamically reconfigurable topology.' Journal of Hydroinformatics 16(6), 1280–1301, doi: 10.2166/hydro.2014.086.

In [ ]:
# existing control valves
prv = ['link_2756', 'link_2729', 'link_2743']
prv_dir = [1, 1, 1]
dbv = ['link_2747', 'link_2748']
iv_open = ['link_2320']
iv_close = ['link_2236', 'link_2682', 'link_2431', 'link_2389', 'link_2508', 'link_2555']

# find connecting nodes
prv_nodes = []
for link_id in prv:
    prv_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())
    
dbv_nodes = []
for link_id in dbv:
    dbv_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())
    
iv_close_nodes = []
for link_id in iv_close:
    iv_close_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())

# plot network layout
legend_labels = {'Inlet (source)': 'black', 'PRV': 'black', 'DBV': 'black', 'IV': 'black'}
plot_network_layout(wdn, prv_nodes=prv_nodes, dbv_nodes=dbv_nodes, iv_nodes=iv_close_nodes, legend_labels=legend_labels)

**Self-cleaning control.** This self-cleaning configuration considers a new design strategy for managing discolouration and pressure management objectives. The design comprises PRVs at new locations and with dynamic set-points at 15-minute intervals. Additionally, automatic flushing valves (AFVs) are installed with dynamic flushing rates at the same frequency. The locations of PRV and AFV locations were optimized using the methods proposed in the following paper:
   * Jenks, B., Ulusoy, A.-J., Pecci, F., and Stoianov, I. (2023). 'Dynamically adaptive networks for integrating optimal pressure management and self-cleaning controls.' Annual Reviews in Control 55, pp. 486–497, doi: 10.1016/j.arcontrol.2023.03.014       

In [ ]:
# self-cleaning control valves
prv_new = ['link_1963', 'link_0749', 'link_0615']
prv_new_dir = [-1, -1, 1]
iv_open = ['link_2320']
iv_close = ['link_2236', 'link_2682', 'link_2431', 'link_2389', 'link_2508', 'link_2555']
afv_nodes = ['node_1285', 'node_1236', 'node_2444', 'node_2095']

# find connecting nodes
prv_nodes = []
for link_id in prv_new:
    prv_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())

# plot network layout
legend_labels = {'Inlet (source)': 'black', 'PRV': 'black', 'AFV': 'black'}
plot_network_layout(wdn, prv_nodes=prv_nodes, afv_nodes=afv_nodes, legend_labels=legend_labels)

### Flow variability metrics
Here, we compute flow variability metrics corresponding to the selected control configuration.

In [ ]:
scenario = 'self-cleaning control' # 'no control', existing control', 'self-cleaning control'

#### Hydraulic simulation
We first update hydraulic controls based on the selected control scenario.

In [ ]:
sim_days_hyd = 1 # number of days in simulation

# call 'set_controls' function
if scenario == 'no control':
    wn = set_controls(net_name, data_path, scenario, iv_close=iv_close+dbv, iv_open=iv_open, sim_days=sim_days_hyd)
        
elif scenario == 'existing control':
    wn = set_controls(net_name, data_path, scenario, iv_close=iv_close, prv=prv, iv_open=iv_open, prv_dir=prv_dir, dbv=dbv, sim_days=sim_days_hyd)
    
elif scenario == 'self-cleaning control':
    wn = set_controls(net_name, data_path, scenario, iv_open=iv_close+iv_open, prv=prv_new, prv_dir=prv_new_dir, afv=afv_nodes, sim_days=sim_days_hyd)

The EPANET solver is then called via `wntr` to compute hydraulic states.

In [ ]:
# 24-hour simulation period with 15-minute time steps
nt = 24
wn.options.time.duration = (nt * sim_days_hyd * 3600) - (0.25 * 3600)
wn.options.time.hydraulic_timestep = 60 * 15
wn.options.time.report_timestep = wn.options.time.hydraulic_timestep
wn.options.time.rule_timestep = wn.options.time.hydraulic_timestep
wn.convert_controls_to_rules(priority=3) 

# run hydraulic solver
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

# obtain flow at links
df_flow = pd.DataFrame()
df_flow['timestamp'] = results.link['flowrate'].index.values / 3600
for col in results.link['flowrate'].columns:
    df_flow[col] = results.link['flowrate'][col].values * 1000
df_flow.set_index('timestamp', inplace=True)

# obtain velocity at links
df_vel = pd.DataFrame()
df_vel['timestamp'] = results.link['velocity'].index.values / 3600
for col in results.link['velocity'].columns:
    df_vel[col] = results.link['velocity'][col].values
df_vel.set_index('timestamp', inplace=True)


# obtain pressure at nodes
df_pressure = pd.DataFrame()
df_pressure['timestamp'] = results.node['pressure'].index.values / 3600
for col in results.node['pressure'].columns:
    df_pressure[col] = results.node['pressure'][col].values
df_pressure.set_index('timestamp', inplace=True)

# obtain head at nodes
df_head = pd.DataFrame()
df_head['timestamp'] = results.node['head'].index.values / 3600
for col in results.node['head'].columns:
    df_head[col] = results.node['head'][col].values
df_head.set_index('timestamp', inplace=True)

# obtain demand at nodes
df_demand = pd.DataFrame()
df_demand['timestamp'] = results.node['demand'].index.values / 3600
for col in results.node['demand'].columns:
    df_demand[col] = results.node['demand'][col].values * 1000
df_demand.set_index('timestamp', inplace=True)

Let's first plot key hydraulic states to ensure the controls are functioning properly.

Hydraulic states across network, including hydraulic head, pressure head, flow rate, flow velocity, etc.

In [ ]:
# plotting time step
t = 38

# network state to plot
state = 'hydraulic head' # 'velocity', 'flow', 'pressure head', 'hydraulic head'
vals_df = df_head.T

# for AZP-SCC control configuration
if scenario == "self-cleaning control":

    # prvs for SCC objective
    prv_nodes = []
    for link_id in prv_new:
        prv_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())
    
    legend_labels = {'Inlet': 'black', 'PRV': 'black', 'AFV': 'black'}
    plot_network_states(wdn, plot_type=state, prv_nodes=prv_nodes, afv_nodes=afv_nodes, vals_df=vals_df, t=t, legend_labels=legend_labels)


# for existing control configuration
elif scenario == "existing control":

    prv_nodes = []
    for link_id in prv:
        prv_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())
        
    dbv_nodes = []
    for link_id in dbv:
        dbv_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())
        
    iv_close_nodes = []
    for link_id in iv_close:
        iv_close_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())

    legend_labels = {'Inlet': 'black', 'PRV': 'black', 'DBV': 'black', 'IV': 'black'}
    plot_network_states(wdn, plot_type=state, prv_nodes=prv_nodes, dbv_nodes=dbv_nodes, iv_nodes=iv_close_nodes, vals_df=vals_df, t=t, legend_labels=legend_labels)

# for no control configuration
elif scenario == "no control":

    # prvs for no control
    prv_nodes = []
    for link_id in prv:
        prv_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())

    dbv_nodes = []
    for link_id in dbv:
        dbv_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())

    iv_close_nodes = []
    for link_id in iv_close:
        iv_close_nodes.extend(wdn.link_df.loc[wdn.link_df['link_ID'] == link_id, ['node_out', 'node_in']].values.flatten())
        
    legend_labels = {'Inlet': 'black', 'PRV': 'black', 'IV': 'black'}
    plot_network_states(wdn, plot_type=state, prv_nodes=prv_nodes, iv_nodes=iv_close_nodes+dbv_nodes, vals_df=vals_df, t=t, legend_labels=legend_labels)

Pressure heads at inlet/outlet of pressure control valves (e.g. PRVs and DBVs).

In [ ]:
# plotting data
inlet = []
outlet = []

if scenario == 'existing control' or scenario == 'no control':
    for idx, link in enumerate(prv):
        if prv_dir[idx] == 1:
            outlet.append(wdn.link_df.loc[wdn.link_df['link_ID'] == link, ['node_in']].to_numpy()[0][0])
            inlet.append(wdn.link_df.loc[wdn.link_df['link_ID'] == link, ['node_out']].to_numpy()[0][0])
        else:
            inlet.append(wdn.link_df.loc[wdn.link_df['link_ID'] == link, ['node_in']].to_numpy()[0][0])
            outlet.append(wdn.link_df.loc[wdn.link_df['link_ID'] == link, ['node_out']].to_numpy()[0][0])
        
elif scenario == 'self-cleaning control':
    for idx, link in enumerate(prv_new):
        if prv_new_dir[idx] == 1:
            outlet.append(wdn.link_df.loc[wdn.link_df['link_ID'] == link, ['node_in']].to_numpy()[0][0])
            inlet.append(wdn.link_df.loc[wdn.link_df['link_ID'] == link, ['node_out']].to_numpy()[0][0])
        else:
            inlet.append(wdn.link_df.loc[wdn.link_df['link_ID'] == link, ['node_in']].to_numpy()[0][0])
            outlet.append(wdn.link_df.loc[wdn.link_df['link_ID'] == link, ['node_out']].to_numpy()[0][0])
        
# plotting code
plt.figure(figsize=(5, 3.75))
df_pressure[outlet].plot(label=outlet, figsize=(5, 3.75))
# df_pressure[inlet].plot(label=inlet)
plt.xlabel('Simulation time [h]', fontsize=12)
plt.ylabel('PRV setpoint [m]', fontsize=12)
plt.tick_params(axis='x', labelsize=11)
plt.tick_params(axis='y', labelsize=11)
leg = plt.legend(loc='upper right', frameon=True, borderpad=0.5, fontsize=11)
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.5)
if sim_days_hyd <= 1:
    plt.xticks(np.arange(0, nt * sim_days_hyd + 1, 6))
elif sim_days_hyd > 1 and sim_days_hyd < 5:
    plt.xticks(np.arange(0, nt * sim_days_hyd + 1, 12))
else:
    plt.xticks(np.arange(0, nt * sim_days_hyd + 1, 24))

# plt.ylim(12-1, 32+1)
# plt.yticks((12, 16, 20, 24, 28, 32))
plt.show()

Time-based DBV loss coefficients.

In [ ]:
if scenario == 'existing control':

    # plotting data
    dbv_setting = pd.read_csv(os.path.join(data_path, 'dbv_exist_settings.csv')).T
    dbv_setting.columns = dbv
    dbv_setting = dbv_setting.div(1000000)

    # plotting code
    dbv_setting.plot(figsize=(5, 3.75))
    plt.xlabel('Simulation time [h]', fontsize=12)
    plt.ylabel('Valve loss coefficient (x $10^6$)', fontsize=12)
    plt.tick_params(axis='x', labelsize=11)
    plt.tick_params(axis='y', labelsize=11)
    leg = plt.legend(loc='upper right', frameon=True, borderpad=0.5, fontsize=11)
    leg.get_frame().set_edgecolor('black')
    leg.get_frame().set_linewidth(0.5)
    plt.xticks(np.arange(0, len(dbv_setting)+1, len(dbv_setting) / 4), labels=np.arange(0, 25, 6))
    plt.yticks((0, 5, 10, 15, 20))
    plt.show()
else:
    print(f'DBVs or kept-shut IVs not included in "{scenario}" scenario.')

Flow across boundary valves (e.g. DBVs and kept-shut BVs).

In [ ]:
if scenario != 'self-cleaning control':
    
    # input plotting data
    link_to_plot = dbv

    # plotting code
    df_flow[link_to_plot].plot(label=link_to_plot, figsize=(5, 3.75))
    plt.xlabel('Simulation time [h]', fontsize=12)
    plt.ylabel('Flow [L/s]', fontsize=12)
    if sim_days_hyd <= 1:
        plt.xticks(np.arange(0, nt * sim_days_hyd + 1, 6))
    elif sim_days_hyd > 1 and sim_days_hyd < 5:
        plt.xticks(np.arange(0, nt * sim_days_hyd + 1, 12))
    else:
        plt.xticks(np.arange(0, nt * sim_days_hyd + 1, 24))
    plt.legend(fontsize=11)
    plt.show()
else:
    print(f'DBVs or kept-shut IVs not included in "{scenario}" scenario.')

Flushing demands at AFV nodes.

In [ ]:
if scenario == 'self-cleaning control':
    
    # input plotting data
    node_to_plot = afv_nodes

    # plotting code
    df_demand[node_to_plot].plot(label=node_to_plot, figsize=(5, 3.75))
    plt.xlabel('Simulation time [h]', fontsize=12)
    plt.ylabel('Flushing demand [L/s]', fontsize=12)
    plt.tick_params(axis='x', labelsize=11)
    plt.tick_params(axis='y', labelsize=11)
    leg = plt.legend(loc='upper right', frameon=True, borderpad=0.5, fontsize=11)
    leg.get_frame().set_edgecolor('black')
    leg.get_frame().set_linewidth(0.5)
    if sim_days_hyd <= 1:
        plt.xticks(np.arange(0, nt * sim_days_hyd + 1, 6))
    elif sim_days_hyd > 1 and sim_days_hyd < 5:
        plt.xticks(np.arange(0, nt * sim_days_hyd + 1, 12))
    else:
        plt.xticks(np.arange(0, nt * sim_days_hyd + 1, 24))
    plt.show()
else:
    print(f'AFVs not included in "{scenario}" scenario.')

#### Source trace simulation
Another flow variability metric we consider is the contribution of source water at nodes across the network. This requires a water quality simulator.

The control settings are updated to extend over a number of simulation days, thereby ensuring water reaches all nodes and periodic behaviour of hydraulic conditions is observed.

In [ ]:
sim_days_qual = 7 # number of days in simulation

# call 'set_controls' function
if scenario == 'no control':
    wn = set_controls(net_name, data_path, scenario, iv_close=iv_close+dbv, sim_days=sim_days_qual)
        
elif scenario == 'existing control':
    wn = set_controls(net_name, data_path, scenario, iv_close=iv_close, prv=prv, prv_dir=prv_dir, dbv=dbv, sim_days=sim_days_qual)
    
elif scenario == 'self-cleaning control':
    wn = set_controls(net_name, data_path, scenario, iv_open=iv_close+iv_open, prv=prv_new, prv_dir=prv_new_dir, afv=afv_nodes, sim_days=sim_days_qual)

The EPANET solver is called via `wntr` to compute water quality (source trace) states.

In [ ]:
# 24-hour simulation period with 15-minute time steps
nt = 24
wn.options.time.duration = (nt * sim_days_qual * 3600) - (0.25 * 3600)
wn.options.time.hydraulic_timestep = 60 * 15
wn.options.time.quality_timestep = 60 * 5
wn.options.time.report_timestep = wn.options.time.hydraulic_timestep
wn.options.time.rule_timestep = wn.options.time.hydraulic_timestep
wn.convert_controls_to_rules(priority=3) 

# set trace parameters
wn.options.quality.parameter = 'TRACE'
sources = wdn.net_info['reservoir_names']
df_trace_0 = pd.DataFrame()
df_trace_1 = pd.DataFrame()

# run water quality solver
for idx, name in enumerate(sources):

    # set trace node
    wn.options.quality.trace_node = name
    sim = wntr.sim.EpanetSimulator(wn)
    results = sim.run_sim()
    
    # obtain quality at nodes
    if idx == 0:
        df_trace_0['timestamp'] = results.node['quality'].index.values / 3600
        for col in results.node['quality'].columns:
            df_trace_0[col] = results.node['quality'][col].values
        df_trace_0.set_index('timestamp', inplace=True)
    elif idx == 1:
        df_trace_1['timestamp'] = results.node['quality'].index.values / 3600
        for col in results.node['quality'].columns:
            df_trace_1[col] = results.node['quality'][col].values
        df_trace_1.set_index('timestamp', inplace=True)

Since '(existing) AZP control' and 'self-cleaning control' configurations allow flow transfer between DMAs, there are a number of nodes which may be supplied from both sources over the simulation period. In this work, we consider such conditions as increased **hydraulic dynamics** in the network. To highlight such areas, we take the maximum average source contribution over the simulation period, whereby nodes with lower values indicate greater **hydraulic dynamics**. Note that the average source contribution is computed across the last day in the simulation to ensure periodic behaviour is observed.

In [ ]:
# compute maximum average source trace over last 24 hours in simulation period
max_avg_trace = pd.concat([df_trace_0.tail(96).mean(), df_trace_1.tail(96).mean()], axis=1).max(axis=1)
df_trace = pd.DataFrame(max_avg_trace, columns=['source_trace']).T

We confirm that periodic behaviour in water quality (source trace) states is observed by plotting select nodes most distant from each source.

In [ ]:
# input plotting data
# node_to_plot = 'node_1827' # north DMA node
# node_to_plot = 'node_0447' # south DMA node
# node_to_plot = 'node_0513' # node at DMA boundary
node_to_plot = 'node_1152' # node at DMA boundary

# plotting code
plt.figure()
df_trace_0[node_to_plot].plot(label=sources[0]+' trace', color='tab:blue')
df_trace_1[node_to_plot].plot(label=sources[1]+' trace', color='tab:red')
plt.xlabel('Simulation time [h]', fontsize=12)
plt.ylabel('Source trace [%]', fontsize=12)
if sim_days_qual <= 1:
    plt.xticks(np.arange(0, nt * sim_days_qual + 1, 6))
elif sim_days_qual > 1 and sim_days_qual < 5:
    plt.xticks(np.arange(0, nt * sim_days_qual + 1, 12))
else:
    plt.xticks(np.arange(0, nt * sim_days_qual + 1, 24))
plt.legend(fontsize=11)
plt.show()

#### Flow variability metric plots
We compute three (3) flow variability metrics to investigate hydraulic dynamics resulting from the considered control scenarios:
* **Flow reversals**: the number of flow direction changes across a link during a 24-h simulation
* **Flow coefficient of variation (CV)**: the relative variability of link flow across a 24-h simulation
* **Mean source trace**: mean source trace percentage across the last 24-h period in a 7-day simulation

These metrics are plotted below for the previously defined control scenario.

In [ ]:
temporal_metric = 'source trace' # 'flow reversal', 'vel cv', 'source trace'
plot_temporal_metric(wdn, temporal_metric, df_flow, df_trace, sensor_names)

### Simulating disinfectant residuals
Here, we simulate disinfectant residuals across BWFLnet for the selected control scenario.

In [ ]:
scenario = 'no control' # 'no control', existing control', 'self-cleaning control'

The control settings are again applied.

In [ ]:
sim_days_qual = 7 # number of days in simulation

# call 'set_controls' function
if scenario == 'no control':
    wn = set_controls(net_name, data_path, scenario, iv_close=iv_close+dbv, iv_open=iv_open, sim_days=sim_days_qual)
        
elif scenario == 'existing control':
    wn = set_controls(net_name, data_path, scenario, iv_close=iv_close, iv_open=iv_open, prv=prv, prv_dir=prv_dir, dbv=dbv, sim_days=sim_days_qual)
    
elif scenario == 'self-cleaning control':
    wn = set_controls(net_name, data_path, scenario, iv_open=iv_open+iv_close, prv=prv_new, prv_dir=prv_new_dir, afv=afv_nodes, sim_days=sim_days_qual)

Water quality simulation parameters are setup in the following code. Note that we apply disinfectant source patterns using sensor data observed at DMA inlets. We also apply bulk and pipe wall decay rates corresponding to recent modelling work in the Field Lab, the results of which are copied into the `get_wall_coeff` function.

In [ ]:
# set disinfectant source patterns

# assign source locations
source_0 = wdn.net_info['reservoir_names'][0]
source_1 = wdn.net_info['reservoir_names'][1]

# assign source values
source_cl = 'worst-case (1)' # 'mean', 'worst-case (1)', worst-case (2)'Any

if source_cl == 'mean':
    # mean inlet source concentrations
    source_0_pat = np.tile(np.random.normal(0.65, 0.15, wdn.net_info['nt']), sim_days_qual)
    source_1_pat = np.tile(np.random.normal(0.75, 0.15, wdn.net_info['nt']), sim_days_qual)

elif source_cl == 'worst-case (1)':
    # worst-case inlet source concentrations (case 1)
    source_0_pat = np.tile(np.random.normal(0.4, 0.15, wdn.net_info['nt']), sim_days_qual)
    source_1_pat = np.tile(np.random.normal(0.95, 0.15, wdn.net_info['nt']), sim_days_qual)

elif source_cl == 'worst-case (2)':
    # worst-case inlet source concentrations (case 2)
    source_0_pat = np.tile(np.random.normal(0.9, 0.15, wdn.net_info['nt']), sim_days_qual)
    source_1_pat = np.tile(np.random.normal(0.55, 0.15, wdn.net_info['nt']), sim_days_qual)

wn.add_pattern("source_0_pat", source_0_pat)
wn.add_source("source_0", source_0, "CONCEN", 1, "source_0_pat")
wn.add_pattern("source_1_pat", source_1_pat)
wn.add_source("source_1", source_1, "CONCEN", 1, "source_1_pat")

In [ ]:
# 24-hour simulation period with 15-minute time steps
nt = 24
wn.options.time.duration = (nt * sim_days_qual * 3600) - (0.25 * 3600)
wn.options.time.hydraulic_timestep = 60 * 15
wn.options.time.quality_timestep = 60 * 5
wn.options.time.report_timestep = wn.options.time.hydraulic_timestep
wn.options.time.pattern_timestep = wn.options.time.hydraulic_timestep
wn.options.time.rule_timestep = wn.options.time.hydraulic_timestep
wn.convert_controls_to_rules(priority=3) 

# setup water quality parameters
wn.options.quality.parameter = "CHEMICAL"
wn.options.reaction.bulk_coeff = (-0.5/3600/24) # units = 1/second
wall_coeff = get_wall_coeff(wdn)
idx = 0
for name, link in wn.links():
    if isinstance(link, wntr.network.Pipe):
        link.wall_coeff = wall_coeff[idx]
    idx += 1

# run water quality solver
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()
df_qual = pd.DataFrame()

# obtain quality (disinfectant) results
df_qual['timestamp'] = results.node['quality'].index.values / 3600
for col in results.node['quality'].columns:
    df_qual[col] = results.node['quality'][col].values
df_qual.set_index('timestamp', inplace=True)

Plot disinfectant source patterns at DMA inlets.

In [ ]:
# plotting code
plt.figure()
df_qual[source_0].plot(label=sources[0], color='tab:blue')
df_qual[source_1].plot(label=sources[1], color='tab:red')
plt.xlabel('Simulation time [h]', fontsize=12)
plt.ylabel('Inlet residual [mg/L]', fontsize=12)
if sim_days_qual <= 1:
    plt.xticks(np.arange(0, nt * sim_days_qual + 1, 6))
elif sim_days_qual > 1 and sim_days_qual < 5:
    plt.xticks(np.arange(0, nt * sim_days_qual + 1, 12))
else:
    plt.xticks(np.arange(0, nt * sim_days_qual + 1, 24))
plt.legend(fontsize=11)
plt.show()

Plot disinfectant residuals across the network at time step $t$.

In [ ]:
last_t = 38
t = 6*96 + last_t
legend_labels = {'Inlet': 'black', 'Sensor': 'black'}
plot_network_states(wdn, plot_type='disinfectant', sensor_nodes=sensor_names, vals_df=df_qual.T, t=t, legend_labels=legend_labels)

Plot disinfectant time series at select model nodes.

In [ ]:
# plotting data
node_to_plot = 'node_0187' # existing control
# node_to_plot = 'node_1900' # existing control
# node_to_plot = 'node_0189' # existing control
node_to_plot = 'node_1149' # existing control
# node_to_plot = 'node_2397' # self-cleaning control
node_to_plot = 'node_2506'

# plotting code
plt.figure()
df_qual[node_to_plot].plot(label=node_to_plot, color='tab:blue')
plt.xlabel('Simulation time [h]', fontsize=12)
plt.ylabel('Disinfectant residual [mg/L]', fontsize=12)
if sim_days_qual <= 1:
    plt.xticks(np.arange(0, nt * sim_days_qual + 1, 6))
elif sim_days_qual > 1 and sim_days_qual < 5:
    plt.xticks(np.arange(0, nt * sim_days_qual + 1, 12))
else:
    plt.xticks(np.arange(0, nt * sim_days_qual + 1, 24))
plt.legend(fontsize=11)
plt.show()

We compute the **standard deviation** (SD) of disinfectant residuals at each node across the last 24-h period of the 7-day simulation. Similar to the flow variability metrics, this aims to highlight nodes with greater disinfectant variability for different control configurations.

In [ ]:
metric = pd.DataFrame(df_qual.tail(wdn.net_info['nt']*(sim_days_qual-1)).std(), columns=['qual_std'])

fig, ax = plt.subplots(figsize=(3.75, 7.25))
ax.margins(0.025, 0.025)

node_weight_name = 'qual_std'

cbar_title = 'Standard deviation [mg/L]'
colorbar_ticks = (np.arange(0, 0.31, 0.1), [str(round(x, 2)) for x in np.arange(0, 0.21, 0.1)] + [r"$\geq 0.3$"])
clims = (0, 0.3)

cmap = cm.get_cmap('RdYlBu').reversed()

# draw network and plot node weights
uG = nx.from_pandas_edgelist(wdn.link_df, source='node_out', target='node_in')
pos = {row['node_ID']: (row['xcoord'], row['ycoord']) for _, row in wdn.node_df.iterrows()}

norm = plt.Normalize(vmin=clims[0], vmax=clims[1])
node_colors = cmap(norm(metric[node_weight_name]))
nx.draw(uG, pos, nodelist=metric.index, node_size=30, node_shape='o', alpha=0.75, linewidths=0, node_color=node_colors, cmap=cmap, edge_color='grey', ax=ax)

# create a color bar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array(metric[node_weight_name])
colorbar = plt.colorbar(sm, orientation='horizontal', pad=-0.025, shrink=0.8)
colorbar.set_label(cbar_title, fontsize=12)
colorbar.set_ticks(colorbar_ticks[0])
colorbar.set_ticklabels(colorbar_ticks[1], fontsize=11)

nx.draw_networkx_nodes(uG, pos, nodelist=wdn.net_info['reservoir_names'], node_size=120, node_shape='s', node_color='black', edgecolors='white') # draw reservoir nodes
nx.draw_networkx_nodes(uG, pos, nodelist=sensor_names, node_size=75, node_shape='o', node_color='black', edgecolors='white') # draw sensor nodes

sensor_labels = {node: str(idx+1) for (idx, node) in enumerate(sensor_names)}
labels_sen = nx.draw_networkx_labels(uG, pos, sensor_labels, font_size=12, verticalalignment='bottom')
for _, label in labels_sen.items():
    label.set_y(label.get_position()[1] + 70)

legend_labels = {'Inlet': 'black', 'Sensor': 'black'}
legend_handles = [plt.Line2D([0], [0], marker='o' if label == 'Sensor' else 's' if label == 'Inlet' else None, markeredgewidth=2, markeredgecolor='white', color='white', markerfacecolor=color, markersize=10 if label == 'Sensor' else 11 if label == 'Inlet' else None, label=label) for label, color in legend_labels.items()]
leg = plt.legend(handles=legend_handles, loc='upper right', frameon=True, borderpad=0.75)
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.5)

### Validation of AZP control configuration modelling through water quality sensor data
We have approximately 3.5 months of continuous (15-minute) water quality data at the highlighted sensor nodes. This data corresponds to the (existing) AZP control configuration. In this section, we aim to validate the increase in disinfectant variability at sensor nodes within the areas of the network with greater hydraulic dynamics. We show this by performing a statistical analysis of sensor time series.

#### Disinfectant variability of raw time series
The standard deviation is computed for each sensor node over the entire monitoring period.

In [ ]:
# load sensor data
wq_data = pd.read_csv(os.path.join(data_path, 'wq_data.csv'))

# delete missing data
wq_data.dropna(how='any', inplace=True)

# plot sensor data
sensor_names = ['node_2746', 'node_1811', 'node_2367', 'node_2747', 'node_1809', 'node_2673', 'node_2506', 'node_1773', 'node_1802']
sensor = 4

plt.figure()
wq_data[sensor_names[sensor-1]].plot(label=f'Sensor {sensor}', color='tab:blue')
plt.xlabel('Time step', fontsize=12)
plt.ylabel('Chlorine [mg/L]', fontsize=12)
plt.legend(fontsize=11)
plt.show()

In [ ]:
# compute standard deviation (SD) and coefficient of variation (CV)
sd = pd.DataFrame(wq_data.std(), columns=['sd'])
cv = pd.DataFrame((wq_data.std() / wq_data.mean()), columns=['cv'])
wq_metrics = sd.merge(cv, left_index=True, right_index=True)

# plot variability metric spatially on sensor node map
legend_labels = {'Sensor node': 'black'}
plot_sensor_data(wdn, sensor_names, wq_metrics[['sd']], legend_labels=legend_labels, sensor_labels=True)

#### Disinfectant variability of differenced (stationary) time series
Remove trend by differencing the time series. This is a common approach for making time series data stationary.

First, we demonstrate the differencing process on time series data of a selected sensor.

In [ ]:
# select sensor time series data
idx = 4
indices = wq_data.index.to_numpy()
sensor_data = wq_data[sensor_names[idx-1]].to_numpy()

# plot initial time series
plt.figure(figsize=(5, 3.75))
plt.plot(np.arange(0, len(sensor_data)), sensor_data, color='tab:blue', label=f'Sensor {idx}')
plt.xlabel('15-minute time step', fontsize=12)
plt.ylabel('Original data [mg/L]', fontsize=12)
plt.tick_params(axis='x', labelsize=11)
plt.tick_params(axis='y', labelsize=11)
leg = plt.legend(loc='upper right', frameon=True, borderpad=0.5, fontsize=11)
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.5)
plt.ylim(0, 1.5)
plt.show()

# compute mean and sd of time series data
mean = np.mean(sensor_data)
print(f'Mean: {mean} mg/L')
sd = np.std(sensor_data)
print(f'Standard deviation (SD): {sd} mg/L')

# check if time series is stationary
result = adfuller(sensor_data)
print(f'ADF statistic: {result[0]}')
print(f'p-value: {result[1]}')
print('Critical values:')
for key, value in result[4].items():
 print(f'{key}: {value}')

# differenced time series
sensor_data_diff = np.diff(sensor_data, n=1)

# delete indices with time step gap
indices_diff = np.diff(indices, n=1)
delete = np.where(indices_diff > 1)
sensor_data_diff = np.delete(sensor_data_diff, delete)

plt.figure(figsize=(5, 3.75))
plt.plot(np.arange(0, len(sensor_data_diff)), sensor_data_diff, color='tab:red', label=f'Sensor {idx}')
plt.xlabel('15-minute time step', fontsize=12)
plt.ylabel('Differenced data [mg/L]', fontsize=12)
plt.tick_params(axis='x', labelsize=11)
plt.tick_params(axis='y', labelsize=11)
leg = plt.legend(loc='upper right', frameon=True, borderpad=0.5, fontsize=11)
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.5)
plt.ylim(-0.5, 0.5)
plt.show()

result = adfuller(sensor_data_diff)
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print(f'usedlag: {result[2]}')

# compute mean of differenced time series data
mean = np.mean(sensor_data_diff)
print(f'Mean: {mean} mg/L')

# compute SD of differenced time series data
sd = np.std(sensor_data_diff)
print(f'Standard deviation (SD): {sd} mg/L')

# compute MAD of differenced time series data
mad = np.mean(np.abs(sensor_data_diff))
print(f'Mean absolute deviation (MAD): {mad} mg/L')

Here, we compute the standard deviation (SD) and mean absolute deviation (MAD) values for each differenced time series data. These values are included in the previously defined `wq_metrics` dataframe.

In [ ]:
wq_metrics[['sd_data', 'mad_data']] = None
sensor_diff_df = pd.DataFrame()

for idx, name in enumerate(sensor_names):
    
    # get time series
    sensor_data = wq_data[name].to_numpy()
    indices = wq_data.index.to_numpy()
    
    # compute differenced time series
    sensor_data_diff = np.diff(sensor_data, n=1)
    indices_diff = np.diff(indices, n=1)
    delete = np.where(indices_diff > 1)
    sensor_data_diff = np.delete(sensor_data_diff, delete)

    # save differenced sensor data to dataframe
    sensor_diff_df[idx+1] = sensor_data_diff
    
    # compute rmse of differenced time series
    sd = np.sqrt(np.mean(sensor_data_diff ** 2))
    wq_metrics['sd_data'][name] = sd
    
    # compute mae of differenced time series
    mad = np.mean(np.abs(sensor_data_diff))
    wq_metrics['mad_data'][name] = mad

Results plotting.

In [ ]:
# SD and MAD metrics bar plot
ax = wq_metrics[['sd_data', 'mad_data']].plot(kind='bar', figsize=(7.4, 4.05), color=['tab:purple', 'tab:green'], legend=True, fontsize=12, alpha=1)
ax.set_xlabel("Sensor ID", fontsize=12)
ax.set_ylabel("Metric value [mg/L]", fontsize=12)
ax.tick_params(axis='x', labelsize=11)
ax.tick_params(axis='y', labelsize=11)
ax.set_xticklabels(np.arange(1, 10))
plt.yticks(np.arange(0, 0.03, 0.005))
plt.xticks(rotation=0)
leg = plt.legend(labels=['SD', 'MAD'], loc='upper right', frameon=False, borderpad=0.75)
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.5)
plt.show()

In [ ]:
histogram_df = pd.DataFrame()
custom_bins = np.linspace(-0.4, 0.4, 9)

# iterate through each column in the original dataframe and compute frequency as percentage
for col_idx in range(sensor_diff_df.shape[1]):

    percentages = pd.cut(sensor_diff_df.iloc[:, col_idx], bins=custom_bins).value_counts() / len(sensor_diff_df) * 100
    percentages = percentages.sort_index(ascending=False)
    percentages = percentages.apply(lambda x: np.nan if np.abs(x) <= 0.01 else x)  # replace 0 values with np.nan
    
    # append the percentages to historgram_df
    histogram_df[col_idx] = percentages

# make custom colorbar
min_val, max_val = 0,1.0
n = 10
orig_cmap = cm.Blues
colors = orig_cmap(np.linspace(min_val, max_val, n))
cmap = mcolors.LinearSegmentedColormap.from_list("mycmap", colors)

norm = plt.Normalize(vmin=0, vmax=100)
    
# plot column heatmap
plt.figure(figsize=(8, 4))
heatmap = sns.heatmap(histogram_df, annot=True, cmap=cmap, fmt=".1f", cbar_kws={'label': 'Frequency [%]', 'norm': norm}, linewidths=1, linecolor='white', annot_kws={"fontsize":11})
heatmap.tick_params(axis='x', labelsize=11)
heatmap.tick_params(axis='y', labelsize=11)
heatmap.set_xticklabels(np.arange(1, 10))
heatmap.collections[0].set_clim(0, 100)
plt.xlabel('Sensor ID', fontsize=12)
plt.ylabel('Differenced data bins [mg/L]', fontsize=12)

# modify colorbar
cbar = heatmap.collections[0].colorbar
cbar.set_label('Frequency [%]', fontsize=12)
cbar.ax.tick_params(labelsize=11)
cbar.set_ticks([0, 20, 40, 60, 80, 100])
cbar.outline.set_edgecolor('black') 
cbar.outline.set_linewidth(1)


for _, spine in heatmap.spines.items():
    spine.set_visible(True)
    spine.set_color('black') 
plt.show()

In [ ]:
# plot variability metric spatially on sensor node map
legend_labels = {'Sensor node': 'black'}
plot_sensor_data(wdn, sensor_names, wq_metrics[['mad_data']], legend_labels=legend_labels, sensor_labels=True)

#### Granger causality (NOT INCLUDED IN PAPER)
In addition to each individual sensor's time series, we also investigate the connectivity between two sensors through the Granger causality test.

First, we demonstrate the granger causality test results for a selected pair of sensors.

In [ ]:
# select sensor indices
idx1 = 2
idx2 = 5

# define sensor data
sensor1_data = wq_data[sensor_names[idx1-1]].to_numpy()
sensor2_data = wq_data[sensor_names[idx2-1]].to_numpy()

# difference sensor data to ensure stationarity
sensor1_diff = np.diff(sensor1_data, n=1).reshape(-1, 1)
sensor2_diff = np.diff(sensor2_data, n=1).reshape(-1, 1)

# define max lag
n_lag = 96

# run granger causality test
data = np.hstack((sensor2_diff, sensor1_diff))
results = grangercausalitytests(data, n_lag, verbose=None)

In [ ]:
# extract F-statistics for each lag
f_statistics = []
for lag in range(1, n_lag + 1):
    f_statistic = results[lag][0]['ssr_ftest'][0]
    f_statistics.append(f_statistic)
    
# ploy F-statistics
plt.plot(np.arange(0, len(f_statistics)), f_statistics, color='tab:blue')
plt.xlabel('Lag', fontsize=12)
plt.ylabel('F-statistic', fontsize=12)
plt.show()

Here, we loop through all sensor pairs to compute all granger causality results.

In [ ]:
# sensor_names = ['node_2746', 'node_1811', 'node_2367', 'node_2747', 'node_1809', 'node_2673', 'node_2506', 'node_1773', 'node_1802'] # edit accordingly
# # sensor_names = ['node_2746', 'node_1811', 'node_2367', 'node_2747', 'node_1809', 'node_2673', 'node_1773', 'node_1802']

# # create an empty DataFrame to store results
# granger_df = pd.DataFrame(columns=['sensor_from', 'sensor_to', 'f_statistic', 'lag'])

# n_lag = 96
# idx = 0
    
# # iterate through each combination of two sensors
# for sensor_from, sensor_to in combinations(sensor_names, 2):
#     for order in [(sensor_from, sensor_to), (sensor_to, sensor_from)]:
    
#         # extract sensor data
#         data_from = wq_data[order[0]].to_numpy()
#         data_to = wq_data[order[1]].to_numpy()

#         # difference sensor data to ensure stationarity
#         data_from_diff = np.diff(data_from, n=1).reshape(-1, 1)
#         data_to_diff = np.diff(data_to, n=1).reshape(-1, 1)

#         # combine the two arrays
#         data = np.hstack((data_to_diff, data_from_diff))

#         # perform Granger causality test
#         results = grangercausalitytests(data, n_lag, verbose=False)

#         # find the lag with the maximum F-statistic
#         max_lag = np.argmax([results[lag][0]['ssr_ftest'][0] for lag in range(1, n_lag + 1)]) + 1
#         max_f_statistic = results[max_lag][0]['ssr_ftest'][0]

#         # append the results to the DataFrame
#         granger_df.loc[idx, ['sensor_from', 'sensor_to', 'f_statistic', 'lag']] = [order[0], order[1], max_f_statistic, max_lag]

#         idx += 1

Make grid plot to illustrate granger causality results.

In [ ]:
# # create pivot table for easy plotting
# pivot_df = granger_df.pivot(index='sensor_from', columns='sensor_to', values='f_statistic')
# pivot_df = pivot_df.reindex(index=sensor_names, columns=sensor_names)

# # create heatmap}
# # plt.figure(figsize=(10, 8))
# # cmap='RdYlBu_r'
# cmap = 'Blues'
# vmin, vmax = 0, 500
# cax = plt.imshow(pivot_df.astype(float), cmap=cmap, interpolation='nearest', vmin=vmin, vmax=vmax)
# plt.xticks(np.arange(9), np.arange(1, 10))
# plt.yticks(np.arange(9), np.arange(1, 10))
# cbar = plt.colorbar(cax, label='F-statistic')
# cbar.set_label('F-statistic', fontsize=12)
# plt.xlabel('Sensor To', fontsize=12)
# plt.ylabel('Sensor From', fontsize=12)
# plt.tick_params(axis='x', which='both', bottom=False, top=True, labeltop=True, labelbottom=False)
# plt.gca().xaxis.set_label_position('top') 
# plt.show()